<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220411_ji_SANUP_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers
# !pip install git+https://github.com/ssut/py-hanspell.git

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 81.9 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 212 kB 71.0 MB/s 
     |████████████████████████████████| 136 kB 81.1 MB/s 
     |████████████████████████████████| 1.1 MB 76.0 MB/s 
     |████████████████████████████████| 127 kB 69.2 MB/s 
     |████████████████████████████████| 271 kB 80.4 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 62.5 MB/s 
     |████████████████████████████████| 596 kB 65.4 MB/s 
     |████████████████████████████████| 895 kB 67.5 MB/s 
     |████████████████████████████████| 6.5 MB 58.6 MB/s 
     |████████████████████████████████| 1.2 MB 72.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher
from pathlib import Path
# from hanspell import spell_checker

- TPU 사용을 위한 accelerate

In [4]:
from accelerate import Accelerator

In [5]:
df = pd.read_csv('/content/drive/MyDrive/220409/train1.csv')

for i in tqdm(range(2, 11)):
  temp = pd.read_csv('/content/drive/MyDrive/220409/train'+str(i)+'.csv')
  df = pd.concat([df,temp], axis=0)

df = df.reset_index(drop=True)

  0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차 부분정비 타이어 오일 교환,S 95 952
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점 내에서 일반인을 대상으로 채소 과일 판매,G 47 472
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여 사업체에도 매 공업용 고무를 가지고 합성고무도 매,G 46 467
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반 소비자에게 열쇠 잠금장치,G 47 475
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학 전 아동보육,Q 87 872
...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,제품 입고 워싱 청바지 워싱,C 13 134
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,현장에서 고객의 요청에 의해 실내 인테리어,F 42 424
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,영업점에서 일반 소비자에게 여성의류 판매,G 47 474
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,사업장에서 일반 고객을 대상으로 필라테스,P 85 856


In [7]:
test = pd.read_csv('/content/drive/MyDrive/220409/test1.csv')

for i in tqdm(range(2, 11)):
  temp = pd.read_csv('/content/drive/MyDrive/220409/test'+str(i)+'.csv')
  test = pd.concat([test,temp], axis=0)
  
test = test.reset_index(drop=True)
test_original = test

  0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
test

,text,target
0,치킨전문점에서 고객의 주문에 의해 치킨 판매,
1,산업공구 다른 소매업자에게 철물 수공구,
2,절에서 신도를 대상으로 불교단체 운영,
3,영업장에서 고객 요구로 자동차 튜닝,
4,실내포장마차에서 접객시설을 갖추고 소주 맥주 제공,
...,...,...
99995,사업장에서 일반인 대상으로 버섯농장,
99996,한의원에서 외래환자 위주고 치료,
99997,일반 점포에서 소비자에게 그림 판매,
99998,사업장에서 일반인 학생 대상으로 학습공간 제공,


In [9]:
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

# 토크나이저, 함수

In [10]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [11]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [12]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## 도 매 -> 도매

In [13]:
def do_mae(text:str):
  try:
    return text.replace('도 매', ' 도매')
  except:
    return text

In [14]:
df['text'] = df['text'].map(do_mae)

In [15]:
test['text'] = test['text'].map(do_mae)

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [16]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220409/digit'+str(digit)+'/model'+str(fold), save_function=accelerator.save)

## digit 별 학습

In [17]:
for digit in range(1,4):
  my_file = Path('/content/drive/MyDrive/220409/digit'+str(digit)+'/model'+str(4))
  if my_file.is_dir():
    continue
  # my_file = Path("/content/drive/MyDrive/220407_inference/digit"+str(i))
  # if my_file.is_dir():
  #   continue
  print('digit'+str(digit))

  #train
  train = df[['text']]
  target = 'digit_'+str(digit)
  train['target'] = df[target]

  my_dict = {}
  for i in range(len(train['target'].unique())):
    my_dict[train['target'].unique()[i]] = i

  def target_to_num(target:str):
    return my_dict[target]

  train['target'] = train['target'].map(target_to_num)
  my_dict['  '] = len(my_dict)

  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

  print('--------------------------------')

  tokenized_train = tokenizer(
    list(train['text']),
    return_tensors="pt",
    max_length=64, # Max_Length = 64
    padding=True,
    truncation=True,
    add_special_tokens=True
    )
  
  for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['target'])):
    my_file = Path('/content/drive/MyDrive/220409/digit'+str(digit)+'/model'+str(fold))
    if my_file.is_dir():
      continue
    print(f'FOLD {fold}')
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    train_label = train['target']
    train_dataset = BERTDataset(tokenized_train, train_label)

    notebook_launcher(training_function)

  #test
  test = test_original
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  test_label = test['target'].map(target_to_num)
  tokenized_test = tokenizer(
      list(test['text']),
      return_tensors="pt",
      max_length=64,
      padding=True,
      truncation=True,
      add_special_tokens=True
  )

  test_dataset = BERTDataset(tokenized_test, test_label)
  dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

  for fold in range(5):
    config = AutoConfig.from_pretrained(checkpoint)
    config.num_labels = len(train['target'].unique())
    model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220409/digit'+str(digit)+'/model'+str(fold), config=config)
    model.resize_token_embeddings(tokenizer.vocab_size)
    accelerator = Accelerator()
    model = accelerator.unwrap_model(model)
    output_pred = []
    output_prob = []
    model, dataloader= accelerator.prepare(model, dataloader)
    model.eval()
    for i, data in enumerate(tqdm(dataloader)):
        with torch.no_grad():
            outputs = model(
                input_ids=data['input_ids'],
                attention_mask=data['attention_mask']
            )
        logits = outputs[0]
        prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)
        output_pred.append(result)
        output_prob.append(prob)
        
    pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
    def num_to_label(label):
        label_dict = {v: k for k, v in my_dict.items()}
        str_label = []
        for i, v in enumerate(label):
            str_label.append([i,label_dict[v]])
        return str_label

    answer = num_to_label(pred_answer)
    df_prob = pd.DataFrame(output_prob)
    df_prob.to_csv('/content/drive/MyDrive/220409/digit'+str(digit)+'/prob_pred'+str(fold)+'.csv', index=False)

digit3
--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

...

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

## 이 밑에 부분 구현
digit1 - prob_pred0, prob_pred1, prob_pred2, prob_pred3, prob_pred4 -> 소프트 보팅 -> digit1 예측

digit2 - prob_pred0, prob_pred1, prob_pred2, prob_pred3, prob_pred4 -> 소프트 보팅 -> digit2 예측

digit3 - prob_pred0, prob_pred1, prob_pred2, prob_pred3, prob_pred4 -> 소프트 보팅 -> digit3 예측

submission -> digit_1, digit_2, digit_3 -> 예측 값으로 교체 -> 제출

### digit_1(soft voting)

In [18]:
pred0 = pd.read_csv('/content/drive/MyDrive/220409/digit1/prob_pred0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220409/digit1/prob_pred1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220409/digit1/prob_pred2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220409/digit1/prob_pred3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220409/digit1/prob_pred4.csv')

In [19]:
pred_1 = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test_1 = pd.concat([test, pred_1], axis=1)
test_1.head(3)

,text,target,0,1,2,3,4,5,6,7,...,9,10,11,12,13,14,15,16,17,18
0,치킨전문점에서 고객의 주문에 의해 치킨 판매,,0.000003,0.000151,1.086548e-06,0.000027,0.999801,4.214171e-07,9.513726e-07,0.000002,...,0.000001,4.293324e-06,4.690397e-07,5.285716e-07,3.409222e-06,1.543410e-07,7.473668e-07,3.211006e-07,8.431416e-07,9.322385e-07
1,산업공구 다른 소매업자에게 철물 수공구,,0.000007,0.999741,4.102984e-07,0.000197,0.000013,3.858416e-08,1.645097e-05,0.000002,...,0.000006,5.794351e-07,2.549168e-06,3.326620e-06,3.408563e-07,2.881177e-07,3.040216e-07,1.932673e-06,1.787214e-07,2.782409e-06
2,절에서 신도를 대상으로 불교단체 운영,,0.999846,0.000039,1.359318e-05,0.000014,0.000009,4.820345e-07,9.106137e-06,0.000011,...,0.000013,1.526289e-05,1.183397e-05,3.249788e-06,4.690920e-06,7.719870e-08,1.552782e-06,4.332795e-07,6.977727e-08,1.812949e-07


In [20]:
train = df[['text']]
target = 'digit_'+str(1)
train['target_1'] = df[target]

my_dict1 = {}
for i in range(len(train['target_1'].unique())):
  my_dict1[train['target_1'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict1[target]

In [21]:
def num_to_label1(label):
    label_dict = label_dict = {v: k for k, v in my_dict1.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label1(np.argmax(np.array(pred_1), axis=-1))

test_1['soft_1'] = pd.DataFrame(answer)[1]

### digit_2

In [22]:
pred0 = pd.read_csv('/content/drive/MyDrive/220409/digit2/prob_pred0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220409/digit2/prob_pred1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220409/digit2/prob_pred2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220409/digit2/prob_pred3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220409/digit2/prob_pred4.csv')

In [23]:
pred_2 = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test_2 = pd.concat([test, pred_2], axis=1)
test_2.head(3)

,text,target,0,1,2,3,4,5,6,7,...,64,65,66,67,68,69,70,71,72,73
0,치킨전문점에서 고객의 주문에 의해 치킨 판매,,8.522811e-08,0.000119,0.000031,5.054565e-07,7.381520e-07,0.999759,0.000045,5.527067e-10,...,1.843215e-07,5.180191e-07,3.501370e-07,6.384418e-08,9.248336e-08,2.673799e-08,2.610626e-08,2.684263e-09,2.700215e-09,1.039178e-08
1,산업공구 다른 소매업자에게 철물 수공구,,7.311699e-06,0.002881,0.996422,3.479509e-07,1.150126e-04,0.000017,0.000008,2.062241e-08,...,1.340605e-06,1.850632e-07,5.224522e-06,7.146348e-09,1.913838e-06,9.910158e-08,1.748224e-07,2.445857e-08,1.716805e-08,4.714008e-08
2,절에서 신도를 대상으로 불교단체 운영,,4.444905e-07,0.000021,0.000015,4.226748e-05,1.304011e-06,0.000011,0.000002,5.851699e-06,...,1.158151e-06,6.069054e-07,3.575782e-07,3.677928e-07,5.096094e-07,7.806196e-08,2.281466e-07,3.842699e-08,3.484463e-09,2.588503e-08


In [24]:
train = df[['text']]
target = 'digit_'+str(2)
train['target_2'] = df[target]

my_dict2 = {}
for i in range(len(train['target_2'].unique())):
  my_dict2[train['target_2'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict2[target]

In [25]:
def num_to_label2(label):
    label_dict = label_dict = {v: k for k, v in my_dict2.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label2(np.argmax(np.array(pred_2), axis=-1))

test_2['soft_2'] = pd.DataFrame(answer)[1]
test_2['soft_2'] = test_2['soft_2'].astype(int)

### digit_3

In [26]:
pred0 = pd.read_csv('/content/drive/MyDrive/220409/digit3/prob_pred0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220409/digit3/prob_pred1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220409/digit3/prob_pred2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220409/digit3/prob_pred3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220409/digit3/prob_pred4.csv')

In [27]:
pred_3 = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test_3 = pd.concat([test, pred_3], axis=1)
test_3.head(3)

,text,target,0,1,2,3,4,5,6,7,...,215,216,217,218,219,220,221,222,223,224
0,치킨전문점에서 고객의 주문에 의해 치킨 판매,,3.098140e-10,0.000204,8.469339e-07,5.520640e-07,9.228917e-08,9.706516e-08,0.998834,3.922251e-05,...,6.809550e-08,4.725217e-09,7.740489e-09,1.726842e-09,2.401944e-10,2.499003e-09,3.800278e-09,7.227974e-09,9.880719e-10,2.016482e-08
1,산업공구 다른 소매업자에게 철물 수공구,,1.093869e-07,0.000001,1.179438e-03,2.616199e-03,1.308256e-07,6.351179e-05,0.000003,3.266219e-07,...,4.949388e-08,2.128554e-08,3.535218e-07,2.048638e-08,1.314077e-08,1.886683e-08,3.826153e-08,1.326272e-07,5.322844e-09,1.623662e-08
2,절에서 신도를 대상으로 불교단체 운영,,3.719081e-06,0.000005,2.193809e-06,2.385217e-07,3.112999e-05,5.370186e-08,0.000002,1.262450e-07,...,8.773662e-09,5.884015e-08,1.676203e-08,5.338720e-09,3.981930e-10,3.663353e-09,9.498501e-09,3.625011e-09,1.309246e-09,1.912139e-09


In [28]:
train = df[['text']]
target = 'digit_'+str(3)
train['target_3'] = df[target]

my_dict3 = {}
for i in range(len(train['target_3'].unique())):
  my_dict3[train['target_3'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict3[target]

In [29]:
def num_to_label3(label):
    label_dict = label_dict = {v: k for k, v in my_dict3.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label3(np.argmax(np.array(pred_3), axis=-1))

test_3['soft_3'] = pd.DataFrame(answer)[1]
test_3['soft_3'] = test_3['soft_3'].astype(int)

# 제출하기

In [30]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [31]:
submission['digit_1'] = test_1['soft_1']

submission['digit_2'] = test_2['soft_2']

submission['digit_3'] = test_3['soft_3']

In [32]:
submission.to_csv('/content/drive/MyDrive/220409/submission.csv', index=False)

In [33]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,C,30,952,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,G,1,11,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86,862,한의원에서,외래환자위주고,치료
99997,id_099998,G,47,478,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90,902,사업장에서,일반인.학생대상으로,학습공간제공
